In [1]:
import os
import re
from pathlib import Path

# Define paths to ruleset directories
BASE_DIR = Path(r"d:\Projects\string-matchers\dataset")
RULESETS_FILE = BASE_DIR / "rulesets.txt"
ET_OPEN_DIR = BASE_DIR / "rulesets" / "et-open" / "rules"
TALOS_DIR = BASE_DIR / "rulesets" / "talos" / "snort3-community-rules"

def extract_content_patterns(rule_text):
    """Extract content patterns from a Snort rule."""
    # Find all content: patterns (quoted strings after content keyword)
    pattern = r'content:"([^"]*)"'
    matches = re.findall(pattern, rule_text)
    return matches

def get_max_rule_length_from_file(filepath):
    """Get the maximum rule length from a single file."""
    max_length = 0
    max_rule = ""
    
    try:
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    length = len(line)
                    if length > max_length:
                        max_length = length
                        max_rule = line[:200]  # Store first 200 chars for display
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
    
    return max_length, max_rule

def get_max_content_pattern_from_file(filepath):
    """Get the maximum content pattern length from a single file."""
    max_pattern_length = 0
    max_pattern = ""
    
    try:
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                if line.strip() and not line.startswith('#'):
                    patterns = extract_content_patterns(line)
                    for pattern in patterns:
                        length = len(pattern)
                        if length > max_pattern_length:
                            max_pattern_length = length
                            max_pattern = pattern
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
    
    return max_pattern_length, max_pattern

# Analyze all rulesets
print("=" * 80)
print("ANALYZING RULESET FILES")
print("=" * 80)

all_rule_files = []

# Add the main rulesets.txt file
if RULESETS_FILE.exists():
    all_rule_files.append(("Main rulesets.txt", RULESETS_FILE))

# Add ET-Open rules
if ET_OPEN_DIR.exists():
    for rule_file in ET_OPEN_DIR.glob("*.rules"):
        all_rule_files.append((f"ET-Open: {rule_file.name}", rule_file))

# Add Talos rules
if TALOS_DIR.exists():
    for rule_file in TALOS_DIR.glob("*.rules"):
        all_rule_files.append((f"Talos: {rule_file.name}", rule_file))

# Track overall maximums
overall_max_rule_length = 0
overall_max_rule_file = ""
overall_max_rule = ""

overall_max_pattern_length = 0
overall_max_pattern_file = ""
overall_max_pattern = ""

# Analyze each file
print(f"\nAnalyzing {len(all_rule_files)} ruleset files...\n")

for name, filepath in all_rule_files:
    max_rule_len, max_rule = get_max_rule_length_from_file(filepath)
    max_pattern_len, max_pattern = get_max_content_pattern_from_file(filepath)
    
    if max_rule_len > overall_max_rule_length:
        overall_max_rule_length = max_rule_len
        overall_max_rule_file = name
        overall_max_rule = max_rule
    
    if max_pattern_len > overall_max_pattern_length:
        overall_max_pattern_length = max_pattern_len
        overall_max_pattern_file = name
        overall_max_pattern = max_pattern

print("\n" + "=" * 80)
print("RESULTS: BIGGEST LENGTHS IN RULESETS")
print("=" * 80)

print(f"\n📏 LONGEST RULE LINE:")
print(f"   File: {overall_max_rule_file}")
print(f"   Length: {overall_max_rule_length} characters")
print(f"   Preview: {overall_max_rule}...")

print(f"\n📏 LONGEST CONTENT PATTERN:")
print(f"   File: {overall_max_pattern_file}")
print(f"   Length: {overall_max_pattern_length} characters")
print(f"   Pattern (first 200 chars): {overall_max_pattern[:200]}")
if len(overall_max_pattern) > 200:
    print(f"   ... ({len(overall_max_pattern) - 200} more characters)")

print("\n" + "=" * 80)

ANALYZING RULESET FILES

Analyzing 49 ruleset files...


RESULTS: BIGGEST LENGTHS IN RULESETS

📏 LONGEST RULE LINE:
   File: Main rulesets.txt
   Length: 3155 characters
   Preview: alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET INFO Java Request to DynDNS Pro Dynamic DNS Domain"; flow:to_server,established; content:" Java/1."; http_header; pcre:"/^Host\x3a\x20[^\r...

📏 LONGEST CONTENT PATTERN:
   File: Main rulesets.txt
   Length: 1549 characters
   Pattern (first 200 chars): |00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 0
   ... (1349 more characters)



In [5]:
# Get first 100 patterns from short_patterns.txt and save to 100_short_patterns.txt
from pathlib import Path

short_patterns_file = Path(r"d:\Projects\string-matchers\dataset\short_patterns.txt")
output_file = Path(r"d:\Projects\string-matchers\dataset\100_short_patterns.txt")

# Read first 100 valid patterns
patterns = []
with open(short_patterns_file, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#'):
            patterns.append(line)
            if len(patterns) >= 100:
                break

# Write to output file
with open(output_file, 'w', encoding='utf-8') as f:
    for pattern in patterns:
        f.write(pattern + '\n')

print(f"Written {len(patterns)} patterns to {output_file}")

Written 100 patterns to d:\Projects\string-matchers\dataset\100_short_patterns.txt
